In [26]:
import numpy as np
import tensorflow as tf
import random

In [27]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import json
import pickle
import warnings
warnings.filterwarnings("ignore")

print("Processing the intents...")

Processing the intents...


In [28]:
with open('/content/sample_data/intents.json') as json_data:
    intents = json.load(json_data)

words = []
classes = []
documents = []
ignore_words = ['?','.','/','!']

In [29]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

46 documents
16 classes ['Identity', 'activity', 'age', 'goodbye', 'greeting', 'haha', 'hours', 'howareyou', 'jokes', 'nicetty', 'opentoday', 'payments', 'programmer', 'thanks', 'weathertoday', 'weathertomorrow']
65 unique stemmed words ["'s", 'a', 'acceiv', 'ag', 'anyon', 'ar', 'be', 'been', 'bye', 'card', 'cash', 'credit', 'day', 'design', 'do', 'doing', 'funny', 'good', 'goodby', 'hah', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'is', 'it', 'jok', 'lat', 'laugh', 'lmao', 'lol', 'mad', 'mak', 'mastercard', 'me', 'nic', 'ok', 'old', 'on', 'op', 'program', 'rofl', 'see', 'tak', 'talk', 'tel', 'tempr', 'thank', 'that', 'ther', 'to', 'today', 'tomorrow', 'upto', 'was', 'weath', 'wer', 'what', 'when', 'who', 'wil', 'yo', 'you']


In [30]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [31]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
tf.compat.v1.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [32]:
import tflearn
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Training....


In [33]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [34]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=350, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('/content/sample_data/model.tflearn')

Training Step: 2099  | total loss: 0.02913 | time: 0.027s
| Adam | epoch: 350 | loss: 0.02913 - acc: 1.0000 -- iter: 40/46
Training Step: 2100  | total loss: 0.02818 | time: 0.029s
| Adam | epoch: 350 | loss: 0.02818 - acc: 1.0000 -- iter: 46/46
--
Saving the Model.......


In [35]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [36]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('/content/sample_data/intents.json') as json_data:
    intents = json.load(json_data)

print("Loading the Model......")
# load our saved model
model.load('/content/sample_data/model.tflearn')

Loading Pickle.....
Loading the Model......


In [37]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer\

You- hi
Good to see you again
You- how are you
I am good
You- who are you?
I am Ella, a chatbot
You- who made you
I was made by Faryal Imran.
You- tell me a joke
As I get older and I remember all the people I’ve lost along the way, I think to myself, maybe a career as a tour guide wasn’t for me.
You- Are you male
I am Ella, a chatbot
You- what your jib
Our hours are 9am-9pm every day
You- what is your owner name 
I was made in 2023, if that's what you are asking!
You- owner
My pleasure
You- creator
My pleasure
You- who is your boss
I was made in 2023, if that's what you are asking!
You- do you like me 
See you later, thanks for visiting
You- are you single
I am Ella, a chatbot
You- who made you
I was made by Faryal Imran.
You- is faryal a cat
My pleasure
You- who is faryal imran
Have a nice day
You- ok give me an appointment of 2am
Glad I could make you laugh!
You- tell me a joke 
A perfectionist walked into a bar...apparently, the bar wasn't set high enough
You- another joke 
What did